<a href="https://colab.research.google.com/github/AnzorGozalishvili/NASA_ODSR_DATA/blob/main/dataset_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bio
!pip install biopython
!pip install boto3 torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.2 MB/s eta 0:00:00


In [3]:
from Bio import SeqIO
import gzip
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import requests
import os
from torch.utils.data import Dataset, DataLoader
import torch

import zipfile
import io
import csv


In [101]:
# !rm -rf downloaded_data

In [102]:
# Initialize S3 client with no-sign-request configuration
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
# dir_list = ["466","465"] #,"524","520"
# Specify your S3 bucket name and directory
bucket_name = 'nasa-osdr'
directory = 'OSD-'

directory_file_dict = {}
directory_list = []

# for i in dir_list:
#   directory = 'OSD-'+i

# List directories under the base directory
response = s3.list_objects(Bucket=bucket_name, Prefix=directory, Delimiter='/')
directories = [common_prefix['Prefix'] for common_prefix in response.get('CommonPrefixes', [])]

# For each directory, list zip files and download them
for directory in directories:
    # List zip files in the directory
    response = s3.list_objects(Bucket=bucket_name, Prefix=directory)
    zip_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.zip') if "metadata" in obj["Key"]]

    # Initialize a list to store filenames for this directory
    directory_filenames = []

    # Download each zip file
    for zip_file_key in zip_files:
        # Extract filename from file key
        filename = zip_file_key.split('/')[-1]

        # Add the filename to the list for this directory
        directory_filenames.append(filename)
        dir = "/".join(zip_file_key.split('/')[:1]) + "/"
        directory_list.append(dir)

        # Create a directory to store downloaded files
        os.makedirs(f'downloaded_data/'+ dir, exist_ok=True)

        # Check if the file already exists
        if os.path.exists(f'downloaded_data/{dir}'+"/"+filename):
            # print(f'{filename} already exists. Skipping...')
            continue
        # Download the zip file
        s3.download_file(bucket_name, zip_file_key, f'downloaded_data/{dir}'+"/"+filename)

    # Add the list of filenames to the dictionary under the directory name
    directory_name = directory.rstrip('/')  # Remove trailing slash
    directory_file_dict[directory_name] = directory_filenames


In [103]:
next(iter(directory_file_dict))

'OSD-1'

In [104]:
directory_list[:10]

['OSD-1/',
 'OSD-100/',
 'OSD-101/',
 'OSD-102/',
 'OSD-103/',
 'OSD-104/',
 'OSD-105/',
 'OSD-106/',
 'OSD-107/',
 'OSD-108/']

In [105]:
for directory in directory_list:
  # Define the directory containing the downloaded zip files
  downloaded_dir = f'downloaded_data/{directory}'
  # Loop through the downloaded zip files
  for filename in os.listdir(downloaded_dir):
      # print(filename)
      if filename.endswith('.txt'):
        continue
      if filename.endswith('.zip'):
          zip_filepath = os.path.join(downloaded_dir, filename)

          # Extract only text files from the specified folder
          with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
              for item in zip_ref.infolist():
                zip_ref.extract(item, downloaded_dir)

          # Remove the zip file after extraction
          os.remove(zip_filepath)
          # print(f'Removed {filename}')
      else:
          continue
      #     # Add the code you want to execute if zip files are found here

In [107]:
from genericpath import exists
import pandas as pd


# Define the delimiter used in the text files
delimiter = '\t'  # For tab-separated files
# found_files = []
for dir in directory_list:
  # Define the root directory
  root_dir = f'downloaded_data/'+ dir  # Replace with the actual root directory
  # Define the file extension you're looking for
  target_extension = '.txt'  # Change to the extension you're interested in

  # List to store found file paths

  # Loop through all directories and subdirectories
  for dirpath, dirnames, filenames in os.walk(root_dir):
      for filename in filenames:
          target_extension = '.txt'
          if filename.endswith(target_extension):
            file_path = os.path.join(dirpath, filename)

            if "investigation" in filename.lower():
              os.remove(file_path)
              continue

            # Read the text file and convert to CSV
            try:
              df = pd.read_csv(file_path, delimiter=delimiter, encoding='cp1252', on_bad_lines='skip')
            except:
              pass

            csv_file = file_path.replace('.txt', '.csv')  # Replace .txt with .csv in the file name
            df.to_csv(csv_file, index=False)
            os.remove(file_path)



In [108]:
!ls downloaded_data/OSD-136/

a_GLDS-136_microarray_metadata.csv  s_GLDS-136_microarray_metadata.csv
i_GLDS-136_microarray_metadata.csv


In [109]:
per_dir_assays = {}
per_dir_samples = {}
for directory in directory_list:
  per_dir_assays[directory] = []
  per_dir_samples[directory] = []

  root_dir = f'downloaded_data/'+ directory

  for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:

      if filename.endswith('.csv'):
        file_path = os.path.join(dirpath, filename)
        try:
          df = pd.read_csv(file_path, encoding='cp1252')
        except:
          df = pd.read_csv(file_path)

        df_columns_lowercase = [str(x.strip().lower()) for x in df.columns]

        if 'source name' in df_columns_lowercase:
          per_dir_assays[directory].append(file_path)

        elif 'sample name' in df_columns_lowercase:
          per_dir_samples[directory].append(file_path)

In [110]:
next(iter(per_dir_samples.items()))

('OSD-1/',
 ['downloaded_data/OSD-1/a_OSD-1_transcription-profiling_dna-microarray.csv'])

In [111]:
pd.Series(per_dir_samples).apply(len).value_counts()

1    374
2     48
3     11
4     10
5      2
6      1
dtype: int64

In [112]:
pd.Series(per_dir_assays).apply(len).value_counts()

1    427
2     18
0      1
dtype: int64

In [118]:
bad_experiment_id = pd.Series(per_dir_assays)[pd.Series(per_dir_assays).apply(len) == 0].index[0]

In [119]:
del per_dir_assays[bad_experiment_id]

In [120]:
len(pd.Series(pd.Series(per_dir_samples).apply(lambda x: [x.strip().lower() for x in pd.read_csv(x[0]).columns]).sum()).unique().tolist())

705

In [121]:
len(pd.Series(pd.Series(per_dir_assays).apply(lambda x: [x.strip().lower() for x in pd.read_csv(x[0]).columns]).sum()).unique().tolist())

787

In [122]:
all_assay_dfs = []
for dir, assay_files in per_dir_assays.items():
  # take first file only!
  df = pd.read_csv(assay_files[0])
  df.columns = [x.strip().lower() for x in df.columns]
  df['osdr_experiment_id'] = dir.rstrip('/')
  all_assay_dfs.append(df)

assays_df = pd.concat(all_assay_dfs)
assays_df.reset_index(drop=True, inplace=True)
assays_df.to_csv('assays.csv')

In [125]:
# all_sample_dfs = []
# for dir, sample_files in per_dir_samples.items():
#   # take first file only!
#   df = pd.read_csv(sample_files[0])
#   df.columns = [x.strip().lower() for x in df.columns]
#   df['osdr_experiment_id'] = dir.rstrip('/')
#   all_sample_dfs.append(df)

# samples_df = pd.concat(all_sample_dfs)
# samples_df.reset_index(drop=True, inplace=True)
# samples_df.to_csv('samples.csv')

In [126]:
# Assays and Samples were different number of records. Need to merge those tables somehow

In [132]:
assays_df.shape

(24188, 788)

In [133]:
assays_df.dtypes

source name                                object
sample name                                object
characteristics[organism]                  object
term source ref                            object
term accession number                      object
                                           ...   
factor value[neoplasm]                     object
characteristics[antibiotic]                object
factor value[antibiotic concentration]    float64
comment[source name- channel 2]            object
factor value[radiation, ionzing]           object
Length: 788, dtype: object

In [135]:
assays_df.dtypes.to_dict()

{'source name': dtype('O'),
 'sample name': dtype('O'),
 'characteristics[organism]': dtype('O'),
 'term source ref': dtype('O'),
 'term accession number': dtype('O'),
 'characteristics[strain]': dtype('O'),
 'term source ref.1': dtype('O'),
 'term accession number.1': dtype('O'),
 'characteristics[sex]': dtype('O'),
 'term source ref.2': dtype('O'),
 'term accession number.2': dtype('O'),
 'characteristics[developmental stage]': dtype('O'),
 'term source ref.3': dtype('O'),
 'term accession number.3': dtype('O'),
 'characteristics[material type]': dtype('O'),
 'term source ref.4': dtype('O'),
 'term accession number.4': dtype('O'),
 'factor value[spaceflight]': dtype('O'),
 'term source ref.5': dtype('O'),
 'term accession number.5': dtype('O'),
 'factor value[infection]': dtype('O'),
 'term source ref.6': dtype('O'),
 'term accession number.6': dtype('O'),
 'characteristics[experiment start date (dd/mm/yyyy)]': dtype('O'),
 'characteristics[experiment end date (dd/mm/yyyy)]': dtype('

In [137]:
import json
with open('assays_dtypes.json', 'w') as file:
  json.dump(assays_df.columns.tolist(), file)